# 1 Import des données

In [2]:
from google.colab import files
import pandas as pd
import numpy as np
from google.colab import autoviz
import seaborn as sns

ModuleNotFoundError: No module named 'google.colab'

In [1]:
files.upload()

NameError: name 'files' is not defined

In [6]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
!cp kaggle.json ~/.kaggle

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
pip install -q kaggle

# Télécharger les  dataset

In [11]:
!kaggle datasets download -d aryansingh16/deepfake-dataset

100% 3.95G/3.96G [03:00<00:00, 22.7MB/s]
100% 3.96G/3.96G [03:00<00:00, 23.6MB/s]


In [12]:
!unzip deepfake-dataset

Streaming output truncated to the last 5000 lines.
  inflating: real_vs_fake/real-vs-fake/valid/real/34825.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34827.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34828.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34832.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34836.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34839.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34847.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34848.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34852.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34861.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34864.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34868.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34877.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34883.jpg  
  inflating: real_vs_fake/real-vs-fake/valid/real/34887.jpg  
  inflating: real_v

In [13]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images

100% 1.68G/1.68G [01:13<00:00, 31.9MB/s]
100% 1.68G/1.68G [01:13<00:00, 24.4MB/s]


In [14]:
!unzip deepfake-and-real-images

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Validation/Real/real_5499.jpg  
  inflating: Dataset/Validation/Real/real_55.jpg  
  inflating: Dataset/Validation/Real/real_550.jpg  
  inflating: Dataset/Validation/Real/real_5500.jpg  
  inflating: Dataset/Validation/Real/real_5501.jpg  
  inflating: Dataset/Validation/Real/real_5502.jpg  
  inflating: Dataset/Validation/Real/real_5503.jpg  
  inflating: Dataset/Validation/Real/real_5504.jpg  
  inflating: Dataset/Validation/Real/real_5505.jpg  
  inflating: Dataset/Validation/Real/real_5506.jpg  
  inflating: Dataset/Validation/Real/real_5507.jpg  
  inflating: Dataset/Validation/Real/real_5508.jpg  
  inflating: Dataset/Validation/Real/real_5509.jpg  
  inflating: Dataset/Validation/Real/real_551.jpg  
  inflating: Dataset/Validation/Real/real_5510.jpg  
  inflating: Dataset/Validation/Real/real_5511.jpg  
  inflating: Dataset/Validation/Real/real_5512.jpg  
  inflating: Dataset/Validation/Real/real_5513.jpg  

# Images generators

Dans cette partie nous générons les images depuis les dataset que nous allons utiliser en test, train et validation

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
# Define image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/real_vs_fake/real-vs-fake/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
validation_generator = test_datagen.flow_from_directory(
    '/content/real_vs_fake/real-vs-fake/valid',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 102041 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [ ]:
import matplotlib.pyplot as plt

# Fonction pour afficher un lot d'images
def plot_images(generator):
    # Obtenir un lot d'images et d'étiquettes
    images, labels = next(generator)
    
    # Définir la taille de la figure
    plt.figure(figsize=(12, 12))
    
    # Afficher les premières 9 images du lot
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(f'Label: {int(labels[i])}')
        plt.axis('off')
    
    plt.show()

# Afficher un lot d'images d'entraînement
plot_images(train_generator)

# Afficher un lot d'images de validation
plot_images(validation_generator)


In [21]:
# Define image data generators
train_datagen2 = ImageDataGenerator(rescale=1./255)
test_datagen2 = ImageDataGenerator(rescale=1./255)

train_generator2 = train_datagen2.flow_from_directory(
    '/content/Dataset/Train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
validation_generator2 = test_datagen2.flow_from_directory(
    '/content/Dataset/Validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


# Définition du modèle



In [18]:
# Define the model
model = tf.keras.Sequential([
  # First convolutional block
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  # Batch normalization for faster convergence (optional)
  # tf.keras.layers.BatchNormalization(),

  # Second convolutional block
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  # Batch normalization for faster convergence (optional)
  # tf.keras.layers.BatchNormalization(),

  # Third convolutional block (added for deeper feature extraction)
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  # Batch normalization for faster convergence (optional)
  # tf.keras.layers.BatchNormalization(),

  # Flatten the feature maps
  tf.keras.layers.Flatten(),

  # Fully-connected layers for classification with dropout
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.5),  # Dropout to prevent overfitting

  tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer with sigmoid activation
])

In [22]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50

)


Epoch 1/15
100/100 [==============================] - 225s 2s/step - loss: 0.7082 - accuracy: 0.5822 - val_loss: 0.6277 - val_accuracy: 0.6569
Epoch 2/15
100/100 [==============================] - 225s 2s/step - loss: 0.5849 - accuracy: 0.6916 - val_loss: 0.5876 - val_accuracy: 0.6781
Epoch 3/15
100/100 [==============================] - 231s 2s/step - loss: 0.5460 - accuracy: 0.7206 - val_loss: 0.5336 - val_accuracy: 0.7312
Epoch 4/15
100/100 [==============================] - 220s 2s/step - loss: 0.4884 - accuracy: 0.7581 - val_loss: 0.5541 - val_accuracy: 0.7275
Epoch 5/15
100/100 [==============================] - 230s 2s/step - loss: 0.4764 - accuracy: 0.7631 - val_loss: 0.5163 - val_accuracy: 0.7444
Epoch 6/15
100/100 [==============================] - 210s 2s/step - loss: 0.4637 - accuracy: 0.7766 - val_loss: 0.4858 - val_accuracy: 0.7487
Epoch 7/15
100/100 [==============================] - 226s 2s/step - loss: 0.4386 - accuracy: 0.7894 - val_loss: 0.4736 - val_accuracy: 0.7794

In [24]:
model.save('mymodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
model1 = tf.keras.models.load_model('/content/mymodel.h5')

In [26]:
history = model1.fit(
    train_generator2,
    steps_per_epoch=100,
    epochs=15,
   validation_data=validation_generator2,
    validation_steps=50
)

Epoch 1/15
100/100 [==============================] - 210s 2s/step - loss: 0.2777 - accuracy: 0.8800 - val_loss: 0.3519 - val_accuracy: 0.8506
Epoch 2/15
100/100 [==============================] - 207s 2s/step - loss: 0.2688 - accuracy: 0.8822 - val_loss: 0.3171 - val_accuracy: 0.8525
Epoch 3/15
100/100 [==============================] - 232s 2s/step - loss: 0.2501 - accuracy: 0.8938 - val_loss: 0.3288 - val_accuracy: 0.8531
Epoch 4/15
100/100 [==============================] - 223s 2s/step - loss: 0.2458 - accuracy: 0.8978 - val_loss: 0.2902 - val_accuracy: 0.8719
Epoch 5/15
100/100 [==============================] - 209s 2s/step - loss: 0.2428 - accuracy: 0.8956 - val_loss: 0.3167 - val_accuracy: 0.8550
Epoch 6/15
100/100 [==============================] - 220s 2s/step - loss: 0.2239 - accuracy: 0.9013 - val_loss: 0.3176 - val_accuracy: 0.8631
Epoch 7/15
100/100 [==============================] - 219s 2s/step - loss: 0.2565 - accuracy: 0.8934 - val_loss: 0.3069 - val_accuracy: 0.8631

In [30]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/Dataset/Test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 10905 images belonging to 2 classes.
341/341 [==============================] - 193s 565ms/step - loss: 0.4632 - accuracy: 0.7892
Test Loss: 0.46323972940444946
Test Accuracy: 0.789179265499115


In [31]:
model.save('mymodel2.h5')

In [33]:
from keras.models import load_model
from keras.preprocessing import image

In [49]:
# Replace 'path/to/your/model.h5' with your actual model path
model_path = '/content/mymodel2.h5'

# Function to load and preprocess image
def prepare_image(img_path):
    # Load image with target dimensions (adjust as per your model)
    img = image.load_img(img_path, target_size=(150, 150))
    # Convert to array and normalize (adjust as per your training process)
    img_array = image.img_to_array(img) / 255.0
    # Expand dimensions for batch prediction (if needed for your model)
    img_array = img_array.reshape((1,) + img_array.shape)
    return img_array

# Load your pre-trained model
model = load_model(model_path)

# Get the path to your input image
input_path = "/content/image8.jpeg"

# Prepare the image
preprocessed_image = prepare_image(input_path)

# Make prediction using the model
predictions = model.predict(preprocessed_image)

# Print the highest probability class
print(predictions)
print(f"Predicted Class: {predictions.argmax(axis=-1)[0]}")

1/1 [==============================] - 0s 117ms/step
[[0.42449602]]
Predicted Class: 0
